# Rectangle selection

In this example, we will use a custom drawing tool to draw rectangle on a 2D figure.
The data inside the rectangles will be summed along the vertical dimension,
and displayed on a one-dimensional plot below the image.

In [ ]:
%matplotlib widget
import plopp as pp
import scipp as sc
import numpy as np
import mpltoolbox as tbx

We first generate some data that contains three bands of peaks that all have different spreads.

In [ ]:
npeaks = 200
per_peak = 500
spread = 30.
ny = 300
nx = 300
a = np.ones((ny, nx))

xbins = np.arange(nx + 1)
ybins = np.arange(ny + 1)
for n in range(npeaks):
    xc = np.random.random() * nx
    yc = np.random.choice([ny/4, ny/2, 3*ny/4])
    xy = np.random.normal(loc=(xc, yc),
                          scale=spread * np.random.random(),
                          size=[per_peak, 2])
    h, ye, xe = np.histogram2d(xy[:, 1], xy[:, 0], bins=(ybins, xbins))
    a += h

da = sc.DataArray(data=sc.array(dims=['y', 'x'], values=a, unit='counts'),
                  coords={'x': sc.array(dims=['x'], values=xbins, unit='cm'),
                          'y': sc.array(dims=['y'], values=ybins, unit='cm')}
                 )

We then construct our custom tool,
using Mpltoolbox's [Rectangles tool](https://mpltoolbox.readthedocs.io/en/latest/rectangles.html),
and inheriting from Plopp's
[DrawingTool](https://scipp.github.io/plopp/about/generated/plopp.widgets.drawing.DrawingTool.html).

In [ ]:
from plopp.widgets.drawing import DrawingTool
from functools import partial
from mpltoolbox import Rectangles

def vertical_sum(da, rect_info):
    """
    Function that slices the data according to the
    rectangle size/position, and sums along the
    vertical dimension.
    """
    x = rect_info['x']
    y = rect_info['y']
    b = min(y['bottom'], y['top'])
    t = max(y['bottom'], y['top'])
    l = min(x['left'], x['right'])
    r = max(x['left'], x['right'])
    return da[y['dim'], b:t][x['dim'], l:r].sum(y['dim'])


def _get_rect_info(artist, figure):
    """
    Convert the raw rectangle info to a dict containing the dimensions of
    each axis, and values with units.
    """
    return lambda: {
        'x': {
            'dim': figure.dims['x'],
            'left': sc.scalar(artist.xy[0], unit=figure.canvas.xunit),
            'right': sc.scalar(artist.xy[0] + artist.width, unit=figure.canvas.xunit)
        },
        'y': {
            'dim': figure.dims['y'],
            'bottom': sc.scalar(artist.xy[1], unit=figure.canvas.yunit),
            'top': sc.scalar(artist.xy[1] + artist.height, unit=figure.canvas.yunit)
        },
    }

RectangleTool = partial(DrawingTool,
                        tool=Rectangles,
                        get_artist_info=_get_rect_info,
                        icon='vector-square')

Finally, we create our visualization interface with two figures,
adding our new tool to the toolbar.

In [ ]:
from plopp.widgets import Box

in_node = pp.input_node(da)
in_node.name = 'Input node'

f2d = pp.figure2d(in_node, norm='log')
f1d = pp.figure1d()

r = RectangleTool(figure=f2d, input_node=in_node, func=vertical_sum, destination=f1d)
f2d.toolbar['roi'] = r

box = Box([f2d, f1d])

In [ ]:
r.value = True

r._tool.click(50, 200)
r._tool.click(200, 250)
r._tool.click(30, 50)
r._tool.click(250, 170)

from ipywidgets import HBox
f1 = box[0]
f1.children = [f1.top_bar, HBox([f1.left_bar, f1.canvas.to_image(), f1.right_bar]), f1.bottom_bar]
f2 = box[1]
f2.children = [f2.top_bar, HBox([f2.left_bar, f2.canvas.to_image(), f2.right_bar]), f2.bottom_bar]

In [ ]:
box

In [ ]:
pp.show_graph(in_node)

In [ ]:
# This cell is used to generate the thumbnail for the docs gallery.
# It is hidden from the online documentation.
f2d.save('../../_static/gallery/rectangle-selection-thumbnail.png')